In [1]:
%matplotlib qt
#%matplotlib inline
import os
import csv
import numpy as np
import re
import datetime
import fnmatch
from pathlib import Path
from itertools import groupby
import copy
import itertools
import distance
import nltk
#nltk.download('stopwords')
import pandas as pd
import matplotlib.pyplot as plt
import pickle

from nltk.stem import SnowballStemmer
from nltk.stem import snowball

from itertools import *
from operator import *

from openpyxl import load_workbook

In [2]:
# exceptional removal of particular extra sentences not typed by the user 
dict_phraseStim = {
    #'2019-02-05-14-10-39_2ndPart_2' : [1, 2, 3, 4, 5, 6, 9, 10],
    #'2019-01-14-14-58-30' : [0], # ys, session_trial ()
    '2019-01-16-16-36-17_1stPart_2' : [-1], # af_session1
    '2019-01-16-17-00-12_2ndPart_2': [1], # af_session1
    '2019-01-17-15-27-20_1stPart_2' : [4], # Af session2
    '2019-01-17-16-03-27_2ndPart_2' : [0, 1, 2], # Af session2
    '2019-02-06-11-25-41_1' : [7],               # aq_session1    
    '2019-02-08-11-33-53_1stPart_1' : [1],  # aq session3_1_part1
    '2019-02-08-12-11-34_2ndPart_1' : [0, 1, 2, 3],  # aq session3_1_part2
    '2019-01-31-09-37-5_2ndPart_2' : range(1,5), # bh1, session 4 , all sentences except the first one deleted
    '2019-01-31-09-22-49_1stPart_2' : [4],  # bh1_session4_2_part1
    '2019-02-21-16-09-44_1stPart_1' : [1], # bh2_session1
    '2019-02-21-16-22-22_2ndPart_1' : [2, 3, 4],# bh2_session1
    '2019-02-28-17-03-53_1stPart_2' : [2],       # bh2_session3
    '2019-02-28-17-24-2_2ndPart_2' : [0, 2],     # bh2_session3
    '2019-02-14-13-28-20_1stPart_2' : [2], # cw_session3_2_part1
    '2019-02-14-13-57-41_2ndPart_2' : [0, 2, 3], # cw_session3_2_part2
    '2019-02-21-15-01-4_1stPart_1' : [0],        # le_session3
    '2019-02-21-15-25-56_2ndPart_1' : [1],        # le_session3
    #'2019-02-18-10-28-35_2' : [0],               # ls2_session4 # picture not described
    '2019-02-05-14-00-27_1stPart_2' : [3],        # mh_session1
    '2019-02-05-14-10-39_2ndPart_2' : [0, 1, 3],   # mh_session1
    '2019-02-08-10-51-3_1stPart_1' : [4],        # mn_session1
    '2019-02-08-11-05-7_2ndPart_1' : [0, 2, 3, 4], # mn_session1
    '2019-02-19-10-34-7_1stPart_1' : [3],          # mn_session3
    '2019-02-19-10-56-43_2ndPart_1' : [1, 2, 3, 4], # mn_session3
    '2019-01-16-15-18-0_1' : [4],            # no_session1
    '2019-02-19-17-10-45_1' : [3],                  # ph_session5
    '2019-01-29-13-25-4_1' : [3],        # ph_session2
    '2019-03-07-16-44-5_2' : [1],                   # rh_session1
    '2019-03-14-13-56-56_2' : [2],                  # rh_session3
    '2019-01-14-15-07-21_1' : [4], # ys_session1
    '2019-01-16-15-18-50_1stPart_1' : [3, 4], # ys_session2
    '2019-01-16-15-42-51_2ndPart_1' : [2], # ys_session2
    '2019-01-30-11-22-25_1' : [3, 5, 7],          # ys_session4
    '2019-01-30-11-22-25_1' : [4, 6, 7] # ys, session 4
}

# exceptional removal of sentences/words typed by the user, but then deleted everything to have a blank scratchpad

dict_phraseUser = {
    "2019-02-06-15-44-15_1" : [2, 3, 6], 
    "2019-02-06-16-19-9_2" : [1, 3, 6, 7],
    "2019-02-12-11-21-21_2" : [0],
    "2019-02-14-14-28-49_1" : [0, 2, 3], # ac_session3_1
    "2019-02-14-14-45-49_2" : [0, 5, 6], # ac_session3_2
    '2019-01-29-14-19-26_1' : [0, 3, 4], # bh1_session2_1
    '2019-01-29-14-40-36_2' : [0, 1, 2], # bh1_session2_2
    '2019-01-30-14-29-29_2' : [4],       # bh1_session3_2
    '2019-01-31-09-12-2_1' : [3],         # bh1_session4_1
    '2019-01-31-09-22-49_1stPart_2' : [4], # bh1_session4_2_part1
    '2019-03-05-09-15-11_1' : [1],         # bh2_session5_1
    '2019-03-05-09-15-11_2' : [1],        # bh2_session5_2
    '2019-02-21-15-55-56_2' : [2],       # ch_session5_2
    '2019-01-30-15-19-36_2' : [1],       # jm_session2_1
    '2019-01-30-15-04-30_1' : [0],         # jm_session2_2
    '2019-01-16-15-18-50_1stPart_1' : [1],  # ys_session2
    '2019-01-16-15-42-51_2ndPart_1' : [0], # ys_session2
    '2019-01-30-11-22-25_1' : [2, 4],       # ys_session4
    '2019-01-30-11-57-3_2' : [0] ,          # ys_session4
    '2019-01-31-13-13-2_1' : [4],           # ys_session5
    '2019-01-30-10-20-32_1' : [0, 1, 2, 3, 4, 5], # no_session4
    '2019-01-30-10-46-38_2' : [0],          # 
    '2019-02-28-17-03-53_1stPart_2' : [2],   # bh2_session3
    '2019-03-12-09-30-5_1' : [0],            # kj_session3
    '2019-02-13-15-20-38_1' : [0, 1, 2, 3, 6], # ls1_session3
    '2019-02-18-10-25-52_1' : [1],              # ls2_session4
    '2019-02-18-10-46-26_2' : [0],            # ls2_session4
    '2019-01-29-13-25-4_1' : [0, 1, 7],        # ph_session2
    '2019-01-29-13-43-50_2' : [0],              # ph_session2
    '2019-03-07-16-17-30_1' : [0],              # rh_session1
    '2019-03-07-16-44-5_2' : [0, 1],         # rh_session1
    '2019-03-14-13-56-56_2' : [0, 1, 3]         # rh_session3
}

# key selection can have extra selections of NextPhrase at the end
dict_keySelectionOfNextPhrase = {
    "2019-02-11-11-18-30_1" : [12, 13], # ac_session1
    "2019-01-16-17-00-12_2ndPart_2" : [12], # af_session1
    "2019-01-17-15-27-20_1stPart_2" : [12], # af_session2
    "2019-02-06-16-19-9_2" : [12], # af_session3
    "2019-02-12-11-07-43_1" : [12], # af_session4
    "2019-02-27-15-08-32_1" : [12], # af_session5
    "2019-01-28-14-30-44_1" : [12], # bh1_session1
    "2019-02-21-16-22-22_2ndPart_1" : [12], # bh2_session1
    "2019-02-18-14-02-56_2" : [12], # le_session1
    "2019-02-19-10-03-14_1" : [12], # le_session2
    "2019-02-08-11-05-7_2ndPart_1" : [12], # mn_session1
    "2019-02-08-11-12-51_2" : [12, 13], # mn_session1
    "2019-02-15-11-38-22_1" : [12, 13], # mn_session2
    "2019-02-15-11-54-25_2" : [12], # mn_session2
    "2019-01-16-15-18-0_1" : [12], # no_session1
    "2019-01-28-13-31-51_1" : [12], # ph_session1
    "2019-01-28-13-49-14_2" : [12], # ph_session1
    "2019-01-14-15-07-21_1" : [12], # ys_session1
    "2019-01-17-15-05-1_1" : [12], # ys_session3
    "2019-01-30-11-22-25_1" : [12], # ys_session4
    "2019-01-31-13-32-2_2" : [12], # ys_session5
}


# key selection when participants skips some sentences
dict_keySelectionNotCompleted = {
    "2019-01-16-16-36-17_1stPart_2" : [0, 1, 3, 5, 7], # af_session1 ---- last sentence is not finished
    "2019-01-16-17-00-12_2ndPart_2" : [0, 1, 3, 4, 5, 7, 9, 11], # af_session1
    "2019-01-17-15-27-20_1stPart_2" : [0, 1, 3, 5, 7, 9, 11], # af_session2 
    "2019-01-17-16-03-27_2ndPart_2" : [0, 1, 2, 3, 4, 5, 6, 7, 9, 11], # af_session2
    "2019-02-08-11-33-53_1stPart_1" : [0, 1, 3, 4, 5, 7, 9, 11], # aq_session3
    "2019-02-08-12-11-34_2ndPart_1" : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11], # aq_session3
    "2019-01-28-14-30-44_1" : [0, 1, 3, 5], # bh1_session1
    "2019-01-31-09-22-49_1stPart_2": [0, 1, 3, 5, 7, 9, 10, 11], # bh1_session4
    "2019-01-31-09-37-5_2ndPart_2" : [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11], # bh1_session4
    "2019-02-21-16-09-44_1stPart_1" : [0, 1, 3, 4, 5, 7, 9, 11], # bh2_session1
    "2019-02-21-16-22-22_2ndPart_1" : [0, 1, 3, 5, 6, 7, 8, 9, 10, 11], # bh2_session1
    "2019-02-28-17-03-53_1stPart_2" : [0, 1, 3, 5, 6, 7, 9, 11], # bh2_session3
    "2019-02-28-17-24-2_2ndPart_2" : [0, 1, 2, 3, 5], # bh2_session3     ----
    "2019-02-14-13-28-20_1stPart_2" : [0, 1, 3, 5, 6, 7, 9, 11], # cw_session3
    "2019-02-14-13-57-41_2ndPart_2" : [0, 1, 2, 3, 5, 6, 7, 8, 9, 11], # cw_session3
    "2019-02-21-15-01-4_1stPart_1" : [0, 1, 2, 3, 5, 7, 9, 11], # le_session3
    "2019-02-21-15-25-56_2ndPart_1" : [0, 1, 3], # le_session3       ----
    "2019-02-05-14-00-27_1stPart_2" : [0, 1, 3, 5, 7, 8], # mh_session1
    "2019-02-05-14-10-39_2ndPart_2" : [0, 1, 2, 3, 4, 5, 7, 8, 9, 11], # mh_session1
    "2019-02-08-10-51-3_1stPart_1" : [0, 1, 3, 5, 7, 9, 10, 11], # mn_session1
    "2019-02-08-11-05-7_2ndPart_1" : [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11], # mn_session1
    "2019-02-19-10-34-7_1stPart_1" : [0, 1, 3, 5, 7, 8, 9, 11], # mn_session3
    "2019-02-19-10-56-43_2ndPart_1" : [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11], # mn_session3
    "2019-01-29-13-25-4_1" : [0, 1, 3, 5, 7], # ph_session2  -- sessions where there are less score questions 
    # and more sentences typed
    "2019-01-16-15-18-50_1stPart_1" : [0, 1, 3, 5, 7, 8, 9, 10], # ys_session2
    "2019-01-17-15-05-1_1" : [0, 1, 3, 5],  # ys_session3  -- sessions where there are less score questions 
    # and more sentences typed
    "2019-02-06-11-25-41_1" : [0, 1, 3, 5, 11], # aq_session1 -- sessions where there are less score questions 
    # and more sentences typed
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 2, 5], # ys_session2 -- different for reading and writing, this one is for
    # writing
    '2019-01-30-11-22-25_1' : [0, 1, 3, 5, 7, 9, 11]   # ys_session4 -- sessions where there are less score questions 
    # and more sentences typed
   
}

# dictionary for phrase removal just like in the dict_phraseStim, but since not all participants require that, some that 
# do, are added to this new dictionary here
dict_keySelection_phraseStim = {
    '2019-01-17-15-27-20_1stPart_2' : [4], # Af session2
    '2019-01-16-15-18-0_1' : [4],        # no_session1
    '2019-02-19-17-10-45_1' : [3],                  # ph_session5
    '2019-03-07-16-44-5_2' : [1],        # rh_session1
    '2019-03-14-13-56-56_2' : [2],              # rh_session3
    '2019-01-14-15-07-21_1' : [4]         # ys_session1
}


# in the beginning experiments, not everyone started with 800 initial dwell time

dict_dwellTimeOrig_not800 = {
    "2019-01-16-15-51-13_2" : 600, # no_session1
    "2019-01-16-15-18-0_1" : 600, # no_session1
    "2019-01-16-15-43-8_1" : 100, # af_session1
    "2019-01-16-16-36-17_1stPart_2" : 100, # af_session1
    "2019-01-16-17-00-12_2ndPart_2" : 100, # af_session1
    "2019-01-17-15-03-40_1" : 100, # af_session2
    "2019-01-17-15-27-20_1stPart_2" : 0, # af_session2
    "2019-01-17-16-03-27_2ndPart_2" : 100, # af_session2
    "2019-01-14-15-07-21_1" : 500, # ys_session1
    "2019-01-14-15-25-55_2" : 300, # ys_session1
    "2019-01-16-15-18-50_1stpart_1" : 200, # ys_session2
    "2019-01-16-15-42-51_2ndPart_1" : 100, # ys_session2
    "2019-01-16-15-59-55_2" : 100, # ys_session2
    "2019-01-17-15-05-1_1" : 100, # ys_session3
    "2019-01-17-15-31-12_2" : 100 # ys_session3
}


# list of all things that should be present when computing effective time
list_keysToBeCounted = ['Comma', 'BackOne', 'BackMany', 'SpaceBar']

# some sessions do not have gaze data
dict_noGazeData = {
    '2019-01-16-17-00-12_2ndPart_2' : 'no gaze data', # af_session2
    '2019-01-17-15-31-12_2' : 'no gaze data', #ys_session2
    '2019-01-30-11-57-3_2' : 'no gaze data' # ys_session4
}



In [3]:
dict_keySelection_ReadingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 3, 5], # ys_session2 
}

dict_keySelection_WritingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 2, 5], # ys_session2   
}

# normally, reading part of trial ends when people look at the keyboardwithphrases. For some trials, this is not done,
# as the reading is done, and the trial is accidentally skipped, and written in the next trial. Here, the trial number 
# given will have the reading time ending as sleep, and not keyboard with phrases. 
dict_keyboardNotChange_ReadingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : 0, # ys_session2 
}

dict_keySelection_firstSleepNotCounted = {
    "2019-01-28-14-50-41_2" : (0, 2), # bh1_session1 -- 3rd sleep activation to be counted
    "2019-02-19-10-56-43_2ndPart_1" : 2  # mn_session3 -- 3rd sleep activation is to be counted
}

In [4]:
TimeDwellOrig = 800
TimeFixation = 300

In [5]:
def FixUserKeys(UserKeys_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    Column_beforeDecimal = [item[2] for item in UserKeys_Old]
    Column_afterDecimal = [item[3] if len(item)>3 else '00' for item in UserKeys_Old]
    
    UserKeys_ProgressPercent = [float(Column_beforeDecimal[i]+'.'+ Column_afterDecimal[i]) for i in 
                                range(0, len(Column_beforeDecimal))]
    UserKeys_Times = [item[0] for item in UserKeys_Old]
    UserKeys_Keys = [item[1] for item in UserKeys_Old]
    
    UserKeys_New = [[UserKeys_Times[ind], UserKeys_Keys[ind], UserKeys_ProgressPercent[ind]] for ind in 
                    range(0, len(UserKeys_ProgressPercent))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    
    return UserKeys_New
        

In [6]:
def FixScratchPad(ScratchPad_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    ScratchPad_Times = [item[0] for item in ScratchPad_Old]
    
    ScratchPad_Phrases = list()
    
    # loop to combine phrases divided by commas
    ScratchPadInd = -1 
    while ScratchPadInd < len(ScratchPad_Old)-1:
        ScratchPadInd = ScratchPadInd + 1
        commasInPhrase = len(ScratchPad_Old[ScratchPadInd])-2
        if commasInPhrase < 1:
            #print(ScratchPad_Old[ScratchPadInd][1])
            ScratchPad_Phrases.append(ScratchPad_Old[ScratchPadInd][1])
            continue
        scratchPadPhrase = ScratchPad_Old[ScratchPadInd][1]
        for phraseJoinNr in range(1, commasInPhrase+1):
            scratchPadPhrase = scratchPadPhrase + ', ' + ScratchPad_Old[ScratchPadInd][1+phraseJoinNr]
        
        ScratchPad_Phrases.append(scratchPadPhrase)
            
        
    ScratchPad_New = [[ScratchPad_Times[ind], ScratchPad_Phrases[ind]] for ind in 
                    range(0, len(ScratchPad_Times))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    #print(ScratchPad_New)
    return ScratchPad_New

In [7]:
def FixKeysSelected(KeysSelected_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    KeysSelected_New = list()
    
    # loop to combine phrases divided by commas
    KeysSelectedInd = -1 
    while KeysSelectedInd < len(KeysSelected_Old)-1:
        KeysSelectedInd = KeysSelectedInd + 1
        
        if KeysSelected_Old[KeysSelectedInd][1].count(',') > 0:
            
            keys_split = KeysSelected_Old[KeysSelectedInd][1].split("\r\n")
            del keys_split[0]
            del keys_split[-1]
            
            keys_split = [key.split(',') for key in keys_split]
            
            KeysSelected_New.extend(keys_split)
        else:
            KeysSelected_New.append(KeysSelected_Old[KeysSelectedInd])
        
    
    return KeysSelected_New

In [8]:
def ComputeDwellTime(userKeys, full_path):
    # modify userKeys to include a column of time instead of progress pct, which is dependent on the then dwell time
    
    TimeDwellOrig = 800
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
    
    if session_folder_name in dict_dwellTimeOrig_not800:
        TimeDwellOrig = dict_dwellTimeOrig_not800[session_folder_name]
    
    #print(TimeDwellOrig)
    
    timeDwell = TimeDwellOrig
    nKey = -1
    for key in userKeys:
        nKey = nKey + 1
        #print(key[1])
        if key[1] == 'IncreaseDwellTime':
            if float(key[2]) == 1:
                timeDwell = timeDwell + 100
        elif key[1] == 'DecreaseDwellTime':
            #print(key[2])
            if float(key[2]) == 1:
                # this is a quick-fix for something weird happening -- two consecutive 'DecreaseDwellTime' selected,
                # but the second one actually does not take effect on the keyboard, and needs to be not counted
                if session_folder_name == '2019-01-17-15-03-40_1' and nKey == 147:
                    continue
                timeDwell = timeDwell - 100
        else:
            userKeys[nKey].append(str(float(key[2])*timeDwell))
    
    return userKeys

In [9]:
def stimPhrasesEdit(PhraseLog, full_path):
   
    # Now extract phrases from the phrase file
    phraseStim_Phrases = [item[1] for item in PhraseLog]
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
        
    phraseStim_PhrasesReduced, phraseStim_timeReduced = zip(*[(x[0], PhraseLog[phraseStim_Phrases.index(x[0])][0]) for x in groupby(phraseStim_Phrases)])
    
    PhraseLogReduced = [[phraseStim_timeReduced[i], phraseStim_PhrasesReduced[i]] for i in range(0, len(phraseStim_PhrasesReduced))]
    
    if PhraseLogReduced[-1][1] == 'THE EXPERIMENT IS NOW DONE':
        del PhraseLogReduced[-1]
        
    if PhraseLogReduced[0][1] == 'phraseText':
        del PhraseLogReduced[0]

    # Here, we want only the sentences typed
    notSentencesToType = list()
    for index in range(0,len(PhraseLogReduced)):
        sentence = PhraseLogReduced[index][1]
        if 'Svar på følgende spørgsmål' in sentence or 'Answer the question:' in sentence or 'What is the complete name of your university?' in sentence or '(give a score between 1 and 7)' in sentence or sentence == '':
            notSentencesToType.append(index)
         
    
    for index in sorted(notSentencesToType, reverse=True):
        del PhraseLogReduced[index]
    
    if session_folder_name in dict_phraseStim:
        #print('session in stim phrases found')
        index_to_be_removed = dict_phraseStim[session_folder_name]
    else:
        index_to_be_removed = []
        
    #print(index_to_be_removed)
    
    if index_to_be_removed:
        for index in sorted(index_to_be_removed, reverse=True):
            del PhraseLogReduced[index]
        
    return PhraseLogReduced

In [10]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        
    return nearestTP, nearestTPind

In [11]:
# function to convert list of date and time into datetime format list
def timeConversion(timeStrList):
    timeList = list()
    for time in timeStrList:
        time1, t1, t2 = time.partition('+')
        timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
    return timeList

In [12]:
def OptiKeyTypingTime(UserKeys):
    
    TimeTyping = dict()
    
    time1, t1, t2 = UserKeys[0][0].partition('+')
    startTime = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    time2, t1, t2 = UserKeys[-1][0].partition('+')
    endTime = datetime.datetime.strptime(re.sub('[:.T]','-',time2[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    TimeTyping['startTime'] = startTime
    TimeTyping['endTime'] = endTime
    
    return TimeTyping

In [13]:
def timeTypingStart(userKeys):
    # From the user keys, find when the user actually starts typing, after having looked at the phrase and all the other 
    # function keys
    
    timeTypingStartInd = 0
    
    timeTypingStartIndList = list()
            
    timeUserTimeInd = 0
    
    ind = 0
    # Get start time of first trial
    
    while ind < len(userKeys):
        #print(len(userKeys[ind][1]))
        if len(userKeys[ind][1]) > 1:
            ind = ind + 1
        else:
            timeTypingStartInd = ind
            timeTypingStartIndList.append(ind)
            break
    
    #print(timeTypingStartInd)
    # Get every next phrase start timings
    while ind < len(userKeys):
        
        if userKeys[ind][1] == 'NextPhrase' and float(userKeys[ind][2]) == 1:
            
            #timeTypingStartIndList.append(ind+1)
            for ind2 in range(ind+1, len(userKeys)):
                if len(userKeys[ind2][1]) > 1:
                    ind = ind + 1
                    continue
                elif userKeys[ind2][1] == 'NextPhrase' and float(userKeys[ind][2]) == 1:
                    ind = ind + 1
                    continue
                else:
                    ind = ind2
                    timeTypingStartIndList.append(ind)
                    break
                    
        else:
            ind = ind + 1
            
    #print(timeTypingStartIndList)
    
    return timeTypingStartIndList

In [14]:
def findAndRemoveTrials(session_name, dictionary_saved, trials, replacingList):
    # function to check the session_name in the dictionary_saved and remove those trials from the dictionary_trial
    
    if session_name in dictionary_saved:
        index_list = dictionary_saved[session_name]
    else:
        index_list = replacingList
    
    
    if index_list:
        if type(trials) == list:
            for index in sorted(index_list, reverse=True):
                del trials[index]
                
        else:
            for index in sorted(index_list, reverse=True):
                del trials['start'][index]
                del trials['end'][index]
        
    return trials    

In [15]:
def scratchPadPhraseEdit(phraseUser, subjName, full_path, picture):
    phraseUserEnd = list()
    
    
    
    for row_ind in range(0, len(phraseUser)):
        if row_ind!= 0 and phraseUser[row_ind][1] == '':
            if phraseUser[row_ind-1][1] != 'scratchPadText':
                if len(phraseUser[row_ind-1][1])>2: # this also removes any answers on the difficulty of the sentence
                    phraseUserEnd.append(phraseUser[row_ind-1])
                    #print(phraseUser[row_ind-1])
                    
    #print(' (1) phrases reduced to :       ', phraseUserEnd)
    # remove first two trials of baseline question and text composition
    if 'not_described' in picture or '2019-01-14-14-58-30' in full_path or '2019-02-06-12-37-45_2' in full_path or '2019-02-18-10-28-35_2' in full_path: 
        # yss has not copied one of the sentences, aq_session1_2 has not written the baseline question, ls2_session4_2 did not describe the picture
        del phraseUserEnd[0]
    else:
        del phraseUserEnd[0:2]
      
    session_folder_name = full_path.split('\\')[-1]
    
    if session_folder_name in dict_phraseUser:
        index_to_be_removed = dict_phraseUser[session_folder_name]
        #print('session in user phrases found')
    else:
        index_to_be_removed = []
        
    
    if index_to_be_removed:
        for index in sorted(index_to_be_removed, reverse=True):
            del phraseUserEnd[index]
        
    
    #print(' (2) phrases reduced to:       ', phraseUserEnd)
    
    # remove the initial rating of complexity, if they have written it:
    for index in range(0,len(phraseUserEnd)):
        sentence = phraseUserEnd[index][1]
        if sentence[0].isdigit():
            
            # if there is also a space after the digit:
            
            if sentence[1] == ' ':
                phraseUserEnd[index][1] = phraseUserEnd[index][1][2:]
            else:
                phraseUserEnd[index][1] = phraseUserEnd[index][1][1:]
            
        elif sentence[1].isdigit():
            if sentence[2] == ' ':
                phraseUserEnd[index][1] = phraseUserEnd[index][1][3:]
            else:
                phraseUserEnd[index][1] = phraseUserEnd[index][1][2:]
    
    #print(' (3) phrases reduced to:       ', phraseUserEnd)
    
    return phraseUserEnd

In [16]:
def FindExptStartEndTimes(KeysSelected, timeTyping, full_path):
    # function to find start and end of tasks in experiments
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
    
    timeTrialDict = dict()
    timeTrialDict = {'start': [],
                    'end':[]}
    
    nTrial = -1
    
    
    
    for keys in KeysSelected:
        
            
        
        if keys[1] == 'NextPhrase':
            nTrial = nTrial + 1
            time1, t1, t2 = keys[0].partition('+')
            endTimeTrial = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            
            if nTrial != 0:
                # print('end: ', endTimeTrial)
                #print('')
                timeTrialDict['end'].append(endTimeTrial)
            
            
            startTimeTrial = endTimeTrial + datetime.timedelta(seconds=5)
            
            #print('start: ', startTimeTrial)
            timeTrialDict['start'].append(startTimeTrial)
        
    del timeTrialDict['start'][-1]
    
    
    # remove the extra selections of NewPhrase at the end of some sessions
    replacingList = []
    timeTrialDict = findAndRemoveTrials(session_folder_name, dict_keySelectionOfNextPhrase, timeTrialDict, replacingList)
    
    timeTrialDict_copy = copy.deepcopy(timeTrialDict)
    
    # separate the reading and writing trials for some participants who read in the actual trial, but write in the next
    # trial
    if session_folder_name in dict_keySelection_ReadingTrials:
        # check the reading and writing separate dictionaries
        print('reading and writing sessions are separate')
        
        #print(len(timeTrialDict['start']))
        # writing trials - 
        timeTrialDict_writing = findAndRemoveTrials(session_folder_name, dict_keySelection_WritingTrials, timeTrialDict, replacingList)
        #print(len(timeTrialDict_copy['start']))
        
        # reading trials
        timeTrialDict_reading = findAndRemoveTrials(session_folder_name, dict_keySelection_ReadingTrials, timeTrialDict_copy, replacingList)
    else:
        # some participants skip some sentences, and then it affects the scoreQuestions too. Remove the skipped sentences or 
        # remove the score questions 
        # for these participants, reading and writing trials are the same
        
        #print('same reading and writing trials')
        scoreQuestions = [0, 1, 3, 5, 7, 9, 11]
        timeTrialDict = findAndRemoveTrials(session_folder_name, dict_keySelectionNotCompleted, timeTrialDict, scoreQuestions)
        
        # most of the skipped sentences are removed, but for those that are not removed
        timeTrialDict_writing = findAndRemoveTrials(session_folder_name, dict_keySelection_phraseStim, timeTrialDict, replacingList)
        
        timeTrialDict_reading = timeTrialDict_writing 
        
    
          
    return timeTrialDict_reading, timeTrialDict_writing 

In [17]:
def FindReadingPartsOfTrial_inKeysSelected(EventTrials_reading, KeysSelected_new, full_path):
    # find the reading end in the pupil size data
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
    
    KeysSelected_timeStr = [key[0] for key in KeysSelected_new]
    KeysSelected_time = timeConversion(KeysSelected_timeStr)
    
    KeysSelected_keys = [key[1] for key in KeysSelected_new]
    
    EventReading = dict()    
    EventReading['start'] = list()
    EventReading['end'] = list()
    
    EventReading_index = dict()    
    EventReading_index['start'] = list()
    EventReading_index['end'] = list()
    
    for ind, startTrialTime_afterCoolDown in enumerate(EventTrials_reading['start']):
        
        startTrialTime, startTrialInd = nearestTimePoint(KeysSelected_time, startTrialTime_afterCoolDown)
        
        EventReading['start'].append(startTrialTime)
        EventReading_index['start'].append(startTrialInd)
        
        endTrialTime = EventTrials_reading['end'][ind]
        endTrialInd = KeysSelected_time.index(endTrialTime)
        
        keysSelected_trial = KeysSelected_keys[startTrialInd:endTrialInd]
        
        # compute the end of reading time -- which is taken when the keyboard with phrases key is pressed
        if session_folder_name not in dict_keyboardNotChange_ReadingTrials:
            
            endReading_keyInd = startTrialInd + keysSelected_trial.index('KeyboardWithPhrases') - 1
            endReading_keyTime = KeysSelected_time[endReading_keyInd]
            
        else:
            if ind == dict_keyboardNotChange_ReadingTrials[session_folder_name]:
                endReading_keyInd = startTrialInd + keysSelected_trial.index('Sleep') - 1
                endReading_keyTime = KeysSelected_time[endReading_keyInd]
                
            else:
                endReading_keyInd = startTrialInd + keysSelected_trial.index('KeyboardWithPhrases') - 1
                endReading_keyTime = KeysSelected_time[endReading_keyInd]
                
                
        EventReading['end'].append(KeysSelected_time[endReading_keyInd+1])
        EventReading_index['end'].append(endReading_keyInd+1)
        
        #print(ind)
        #print('reading: ', EventReading['start'][ind], EventReading['end'][ind])
        
    
    
    return EventReading, EventReading_index

In [18]:
def FindWritingPartsOfTrial_inKeysSelected(EventTrials_writing, KeysSelected_new, EventReading):
    
    KeysSelected_timeStr = [key[0] for key in KeysSelected_new]
    KeysSelected_time = timeConversion(KeysSelected_timeStr)
    
    KeysSelected_keys = [key[1] for key in KeysSelected_new]
    
    EventWriting = dict()    
    EventWriting['start'] = list()
    EventWriting['end'] = list()
    
    EventWriting_index = dict()    
    EventWriting_index['start'] = list()
    EventWriting_index['end'] = list()
    
    for ind, startTrialTime_afterCoolDown in enumerate(EventTrials_writing['start']):
        
        startTrialTime, startTrialInd = nearestTimePoint(KeysSelected_time, startTrialTime_afterCoolDown)
        
        endTrialTime = EventTrials_writing['end'][ind]
        endTimeReading = EventReading['end'][ind]
        
        
        # for some participants, reading and writing trials are different. So their writing times will not be the end of 
        # the reading time.
        # Regardless, the writing time should start later than when the reading time ends.
        # We choose the starting time for writing as the one that is later than the start time from writing trials
        # and end time from reading trials
        
        if startTrialTime > endTimeReading:
            EventWriting['start'].append(startTrialTime)
            EventWriting_index['start'].append(startTrialInd)
        else:
            EventWriting['start'].append(endTimeReading)
            EventWriting_index['start'].append(KeysSelected_time.index(endTimeReading))
        
        EventWriting['end'].append(endTrialTime)
        EventWriting_index['end'].append(KeysSelected_time.index(endTrialTime))
        
        #print(ind)
        #print('writing: ', EventWriting['start'][ind], EventWriting['end'][ind])
        
    
    
    return EventWriting, EventWriting_index     

In [20]:
def CharactersTyped(EventWriting, KeysSelected_new):
    # total number of letters typed, 
    
    KeysSelected_timeStr = [key[0] for key in KeysSelected_new]
    KeysSelected_time = timeConversion(KeysSelected_timeStr)
    
    KeysSelected_keys = [key[1] for key in KeysSelected_new]
    
    nCharacters = list() 
    
    for i, timeStart in enumerate(EventWriting['start']):
        
        timeStartKey, timeStartKeyInd = nearestTimePoint(KeysSelected_time, timeStart)
        timeEndKey, timeEndKeyInd = nearestTimePoint(KeysSelected_time, EventWriting['end'][i])
        
        
        # remove keys that should not be counted towards the Characters - the score given before writing the sentence
        # till the characterKeyboard is selected, nothing to be selected till then
        nNotCharacters = 0
        
        for ind, key in enumerate(KeysSelected_keys[timeStartKeyInd-1:timeEndKeyInd+1]):
            if key == 'KeyboardWithPhrases':
                keyNewStart = ind+1
                break
            
        for ind, key in enumerate(KeysSelected_keys[timeStartKeyInd+keyNewStart-1:timeEndKeyInd+1]):
            if key == 'ScratchPad' or key == 'PhraseTextBlock' or key == 'BackSpace' or key == 'BackMany' or key == 'LeftShift' or key == 'Sleep' or 'Keyboard' in key:
                nNotCharacters = nNotCharacters + 1
        
        KeysSelected_trial_writingSentence = KeysSelected_keys[timeStartKeyInd+keyNewStart-1:timeEndKeyInd+1]
        KeysSelected_trial_time = KeysSelected_time[timeStartKeyInd+keyNewStart-1:timeEndKeyInd+1]
        
        #print(KeysSelected_trial_time[0], KeysSelected_trial_writingSentence[0])
        
        #print(KeysSelected_trial_time[-1], KeysSelected_trial_writingSentence[-1])
        
        #print('')
        
        
        nCharacters.append(len(KeysSelected_trial_writingSentence)-nNotCharacters)
        print(len(KeysSelected_trial_writingSentence)-nNotCharacters)
    
    return nCharacters

In [21]:
def EffectiveTimeFromUserKeys(UserKeys, Event, pathOfSession):
    
    UserKeysTimeStr = [val[0] for val in UserKeys]
    UserKeysTime = timeConversion(UserKeysTimeStr)
    
    timeActivation = 250
    
    # session name
    session_folder_name = pathOfSession.split('\\')[-1]
    
    timeTypingList = list()
    
    
    for indTrial, eventStart in enumerate(Event['start']):
        eventEnd = Event['end'][indTrial]
        eventStart_UserKeys, eventStartUserKeysInd = nearestTimePoint(UserKeysTime, eventStart)
        eventEnd_UserKeys, eventEnd_UserKeysInd = nearestTimePoint(UserKeysTime, eventEnd)
        time_trial = 0
        
        keySelectedPrevious = ''
        
        for indUserKey in range(eventStartUserKeysInd, eventEnd_UserKeysInd):
            
            if len(UserKeys[indUserKey][1]) == 1 or UserKeys[indUserKey][1] in list_keysToBeCounted or 'Suggestion' in UserKeys[indUserKey][1]:
                
                if len(UserKeys[indUserKey+1]) < 4 or float(UserKeys[indUserKey][3]) > float(UserKeys[indUserKey+1][3]) or UserKeys[indUserKey][2]==1:
                    
                    #print(UserKeys[indUserKey])
                    
                        
                    # Sometimes, in between selection of a letter, there is a small pause and activation of another letter,
                    # indicated by a '0' activation of the second letter during the first letter selection.
                    # This requires adding only the activation time for the second letter, not the dwell-time for the 
                    # current letter, which will be completed later on
                    
                    if len(UserKeys[indUserKey+1])>3 and len(UserKeys[indUserKey+2])>3:
                        if float(UserKeys[indUserKey][3]) > float(UserKeys[indUserKey+1][3]):
                            if float(UserKeys[indUserKey+1][3]) == 0:
                                if UserKeys[indUserKey+2][1] == UserKeys[indUserKey][1]:
                                    time_trial = time_trial + timeActivation
                                    #print('Only timeActivation added')
                                    if UserKeys[indUserKey][2]==1:
                                        keySelectedPrevious = UserKeys[indUserKey][1]
                                    continue
                    
                    if keySelectedPrevious == UserKeys[indUserKey][1]: # if the key selection is accidentally continued,
                        # activation time should not be counted
                        time_trial = time_trial + float(UserKeys[indUserKey][3])
                        #print('Current dwell-time added', keySelectedPrevious)
                        
                    else:
                        #print('timeActivation added')
                        time_trial = time_trial + float(UserKeys[indUserKey][3]) + timeActivation
                    
                    if UserKeys[indUserKey][2]==1:
                            keySelectedPrevious = UserKeys[indUserKey][1]
        
        print(time_trial/1000)                    
        timeTypingList.append(time_trial/1000) # add the time in seconds
        
    
    return timeTypingList

In [26]:
def TimeBetweenKeys(userKeys):
    
    timeBetweenKeysList = list()
    
    keyPrevious = userKeys[0]
    
    for i, userKey in enumerate(userKeys):
        
        
        
        
        # Proceeding here means keyPrevious and keyCurrent aren't the same
        if len(keyPrevious[1]) > 1: # it is not a letter
            keyPrevious = userKey
            continue
        
        if len(userKey[1]) > 1: # it is not a letter
            keyPrevious = userKey
            continue
        
        if float(keyPrevious[2]) < 1:
            keyPrevious = userKey
            continue
        
        if keyPrevious[1] == userKey[1]:
            continue
        
        #print(i, keyPrevious, userKey)
        
        time1, t1, t2 = userKey[0].partition('+')
        timeCurrent = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
        
        time1, t1, t2 = keyPrevious[0].partition('+')
        timePrevious = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
        
        timeBetweenKeysList.append([timeCurrent, (timeCurrent - timePrevious - datetime.timedelta(milliseconds = \
            float(userKey[3])) - datetime.timedelta(milliseconds = 250)).total_seconds()])
        
        keyPrevious = userKey   
        
        
    return timeBetweenKeysList  

In [27]:
class DataForEverySubject:
    
    timeStamp = ''
    timeBetweenKeys = ''
    
    resultPathName = ''
   
    
    def printInfo(self):
        dataFrame = pd.DataFrame(list(zip(self.timeStamp, self.timeBetweenKeys)), columns=['timeStamp', 'timeBetweenKeys'])
        
        return dataFrame
    
    def AddToFile(self):
        
        dataFrame = pd.DataFrame(list(zip(self.timeStamp, self.timeBetweenKeys)), columns=['timeStamp', 'timeBetweenKeys'])
        
        #workbook = xlsxwriter.Workbook(self.resultPathName)
        #book = workbook.add_worksheet()
        
        #book = load_workbook(self.resultPathName)
        writer = pd.ExcelWriter(self.resultPathName, engine='xlsxwriter')
        #writer.book = book
        #writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
        
        dataFrame.to_excel(writer, index = False, header = False)
        workbook  = writer.book
        worksheet = writer.sheets['Sheet1']
        writer.save()
    
    

In [29]:

dataFolderName = r'E:\Data\Data' # accessing external hard disk with the data
a = re.compile('(?<=Data\\\\Data\\\\)(.*)(?=\\\\[1-9])')
subjectName_listElement = 3

#dataFolderName = r'C:\DTU\Data\201901_JanuaryExpt' # accessing data saved in the computer
#a = re.compile('(?<=Data\\\\201901_JanuaryExpt\\\\)(.*)(?=\\\\[1-9])')
#subjectName_listElement = 4



for root, dirs, subfolder in os.walk(dataFolderName):
    
    technique = 'dwell_time'
    
    if not dirs:
        
        #if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
        
        if 'noData' in root or 'Trial' in root or 'trial' in root or 'Nothing' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'Jonas' in root or 'Praktikant' in root or 'Villads' in root:
            continue
            
        #if 'af\\1' not in root:
        #    continue
        if 'Picture' in root:
            continue
        #if '_MS' not in root:
        #    continue
        
        
            
        keysSelected = None
        userKeys = None
        phraseLog = None
        
        for file in subfolder:
            
            if fnmatch.fnmatch(file, 'ScratchPadLog*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerScratchPad = csv.reader(fScratchPad, quotechar=None)
                    scratchPad = list(readerScratchPad)
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the scratchpad log file')
            
            if fnmatch.fnmatch(file, 'KeySelection*'):
                try:
                    
                    fKeysSelected = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerKeysSelected = csv.reader(fKeysSelected)
                    keysSelected = list(readerKeysSelected)
                    
                    keysSelected.remove(keysSelected[0])
                except:
                    if fKeysSelected is not None:
                        fKeysSelected.close()
                    else:
                        print('error in opening the KeySelection log file')
                        
            
            if fnmatch.fnmatch(file, 'user*'):
                try:
                    fUserKey = open(root + '\\' + file, encoding='utf-8',  newline='')
                    readerUserKey = csv.reader(fUserKey, quotechar=None)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user key log file')
                        
            if fnmatch.fnmatch(file, 'phrase*'):
                try:
                    fPhraseLog = open(root + '\\' + file, encoding='utf-8')
                    readerPhraseLog = csv.reader(fPhraseLog, quotechar=None)
                    phraseLog = list(readerPhraseLog)
                    
                except:
                    if fPhraseLog is not None:
                        fPhraseLog.close()
                    else:
                        print('error in opening the phrase log file')
                       
            if fnmatch.fnmatch(file, 'multiKey*'):
                technique = 'multiKey_selection'
            
            if fnmatch.fnmatch(file, 'tobiiGazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerGazeLog = csv.reader(fGazeLog, quotechar=None)
                    gazeLog = list(readerGazeLog)
                    gazeLog.remove(gazeLog[0])
                    gazeLog.remove(gazeLog[-1])
                    
                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the scratchpad log file')
            
                    
                     
        if keysSelected is None or userKeys is None or phraseLog is None or gazeLog is None or scratchPad is None:
            continue
        else:
                
            print('subject path', root)
            subjAndSessionName = '__'.join(root.split('\\')[subjectName_listElement:])
            subjName = subjAndSessionName.split('__')[0]
            print('subject and session name: ', subjAndSessionName)
            sessionFolderName = root.split('\\')[-1]
            
            resultFileName = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\Subjects\Performance' + '\\' + subjName + \
            '\\' + subjAndSessionName +  '.xlsx'
            
            # fix scratchpad due to comma related file changes
            scratchPad_new = FixScratchPad(scratchPad)
            
            # fix phraselog due to comma related file changes
            phraseLog_new = FixScratchPad(phraseLog)
            
            # fix userKeys due to comma related file changes
            userKeys_new = FixUserKeys(userKeys)
            
            # find dwell time of typing
            userKeys_wDwellTime = ComputeDwellTime(userKeys_new, root)
            
            
            # need to fix keys selected, where rows get combined because of an inverted comma
            keysSelected_new = FixKeysSelected(keysSelected)
            
            # find time between selection of 1 key and start of selection of the next
            timeBetweenKeysList = TimeBetweenKeys(userKeys_wDwellTime)
            
            
            timeList = [key[0] for key in timeBetweenKeysList]
            timeBetweenKeys = [key[1] for key in timeBetweenKeysList]
            
            if '1stPart' in root:
                print('1stPart')
                timeList1 = timeList
                timeBetweenKeys1 = timeBetweenKeys
                continue
            
            
            
            if '2ndPart' in root:
                print('2ndPart')
                timeList2 = timeList
                timeBetweenKeys2 = timeBetweenKeys
                
                timeList = timeList1 + timeList2
                timeBetweenKeys = timeBetweenKeys1 + timeBetweenKeys2
                
                timeList1 = list()
                timeBetweenKeys1 = list()
                
                
            dataToSave = DataForEverySubject()
            dataToSave.timeStamp = timeList
            dataToSave.timeBetweenKeys = timeBetweenKeys
            dataToSave.resultPathName = resultFileName
            
            print(dataToSave.printInfo())
            
            
            dataToSave.AddToFile()
            
            
            
            
            

subject path E:\Data\Data\ac\1\2019-02-11-11-18-30_1
subject and session name:  ac__1__2019-02-11-11-18-30_1
                     timeStamp  timeBetweenKeys
0   2019-02-11 11:18:38.624506         0.572303
1   2019-02-11 11:18:40.156833         0.472327
2   2019-02-11 11:19:21.652120         0.415499
3   2019-02-11 11:19:23.473242         0.595025
4   2019-02-11 11:19:24.872099         0.339802
5   2019-02-11 11:19:26.227517         0.295207
6   2019-02-11 11:19:27.569358         0.283408
7   2019-02-11 11:19:32.089923         0.294453
8   2019-02-11 11:19:33.732137         0.582421
9   2019-02-11 11:19:36.064380         0.827034
10  2019-02-11 11:19:37.452851         0.328543
11  2019-02-11 11:19:38.838857         0.326923
12  2019-02-11 11:19:40.727921         0.328137
13  2019-02-11 11:19:45.202003         0.504960
14  2019-02-11 11:19:51.720059         0.183732
15  2019-02-11 11:19:52.919193         0.349997
16  2019-02-11 11:19:54.629645         0.848731
17  2019-02-11 11:19:55.794

[448 rows x 2 columns]
subject path E:\Data\Data\ac\2\2019-02-12-15-27-37_2
subject and session name:  ac__2__2019-02-12-15-27-37_2
                     timeStamp  timeBetweenKeys
0   2019-02-12 15:27:46.049147         0.126117
1   2019-02-12 15:27:47.302434         0.194154
2   2019-02-12 15:28:19.752411         0.237571
3   2019-02-12 15:28:20.673020         0.273244
4   2019-02-12 15:28:21.862109         0.527815
5   2019-02-12 15:28:22.770922         0.259860
6   2019-02-12 15:28:23.704742         0.284632
7   2019-02-12 15:28:25.269292         0.538433
8   2019-02-12 15:28:26.168642         0.238927
9   2019-02-12 15:28:33.586603         0.404550
10  2019-02-12 15:28:37.727582         0.262002
11  2019-02-12 15:28:43.613331         0.495416
12  2019-02-12 15:28:44.512511         0.239629
13  2019-02-12 15:28:45.378233         0.206258
14  2019-02-12 15:28:46.355203         0.317708
15  2019-02-12 15:28:50.255453         0.193407
16  2019-02-12 15:28:51.233958         0.316480
17  

subject path E:\Data\Data\ac\4\2019-02-18-14-59-21_1
subject and session name:  ac__4__2019-02-18-14-59-21_1
                     timeStamp  timeBetweenKeys
0   2019-02-18 14:59:29.564057         0.227026
1   2019-02-18 14:59:39.656838         0.282601
2   2019-02-18 14:59:41.312146         0.493672
3   2019-02-18 14:59:42.255081         0.192718
4   2019-02-18 14:59:43.211771         0.195851
5   2019-02-18 14:59:44.309388         0.337072
6   2019-02-18 14:59:45.286172         0.227775
7   2019-02-18 14:59:46.276066         0.227702
8   2019-02-18 14:59:47.508230         0.472057
9   2019-02-18 14:59:52.316373         0.174378
10  2019-02-18 14:59:53.447471         0.371390
11  2019-02-18 14:59:54.380445         0.183133
12  2019-02-18 14:59:55.368676         0.227844
13  2019-02-18 14:59:56.312452         0.183751
14  2019-02-18 14:59:57.248612         0.175765
15  2019-02-18 14:59:58.414376         0.405866
16  2019-02-18 14:59:59.613357         0.439518
17  2019-02-18 15:00:00.579

subject path E:\Data\Data\ac\5\2019-02-19-15-18-20_2
subject and session name:  ac__5__2019-02-19-15-18-20_2
                     timeStamp  timeBetweenKeys
0   2019-02-19 15:18:34.771471         0.495225
1   2019-02-19 15:18:35.447970         0.226515
2   2019-02-19 15:18:36.036721         0.139730
3   2019-02-19 15:18:36.746711         0.259231
4   2019-02-19 15:18:37.401519         0.204861
5   2019-02-19 15:18:40.378504         0.202806
6   2019-02-19 15:18:41.012726         0.184975
7   2019-02-19 15:18:41.631806         0.169697
8   2019-02-19 15:18:42.264774         0.182777
9   2019-02-19 15:18:43.818833         1.104416
10  2019-02-19 15:18:44.919412         0.161899
11  2019-02-19 15:18:46.507043         1.137107
12  2019-02-19 15:18:48.694553         0.160830
13  2019-02-19 15:18:51.391723         0.172005
14  2019-02-19 15:18:54.189483         0.149119
15  2019-02-19 15:18:54.989426         0.339049
16  2019-02-19 15:18:55.645697         0.194845
17  2019-02-19 15:18:58.176

subject path E:\Data\Data\af\2\2019-01-17-15-03-40_1
subject and session name:  af__2__2019-01-17-15-03-40_1
                     timeStamp  timeBetweenKeys
0   2019-01-17 15:03:54.947742         0.349397
1   2019-01-17 15:04:14.970563         0.317481
2   2019-01-17 15:04:15.602475         0.381912
3   2019-01-17 15:04:16.170032         0.317557
4   2019-01-17 15:04:18.923399         0.438666
5   2019-01-17 15:04:19.558209         0.384810
6   2019-01-17 15:04:20.123207         0.314998
7   2019-01-17 15:04:20.732576         0.359369
8   2019-01-17 15:04:21.465269         0.482693
9   2019-01-17 15:04:23.995979         0.325009
10  2019-01-17 15:04:24.596372         0.350393
11  2019-01-17 15:04:25.118509         0.272137
12  2019-01-17 15:04:25.762297         0.393788
13  2019-01-17 15:04:31.448010         0.272603
14  2019-01-17 15:04:32.248001         0.549991
15  2019-01-17 15:04:34.567807         0.361719
16  2019-01-17 15:04:39.252907         0.350230
17  2019-01-17 15:04:39.886

...

subject path E:\Data\Data\rh\3_MS\2019-03-14-13-56-56_2
subject and session name:  rh__3_MS__2019-03-14-13-56-56_2
                     timeStamp  timeBetweenKeys
0   2019-03-14 13:57:08.687554         0.394030
1   2019-03-14 13:57:43.286550         2.604821
2   2019-03-14 13:57:48.796083         0.236480
3   2019-03-14 13:57:57.500717         0.239228
4   2019-03-14 13:58:05.227813         0.215823
5   2019-03-14 13:58:40.781959         0.228213
6   2019-03-14 13:58:48.398643         0.149647
7   2019-03-14 13:58:55.284527         0.224837
8   2019-03-14 13:59:02.292314         0.216408
9   2019-03-14 13:59:08.165117         0.160495
10  2019-03-14 13:59:11.795540         0.126766
11  2019-03-14 13:59:25.064763         0.351726
12  2019-03-14 13:59:30.071727         0.224192
13  2019-03-14 13:59:33.069678         0.038289
14  2019-03-14 13:59:37.222075         0.826644
15  2019-03-14 13:59:38.620923         0.348644
16  2019-03-14 13:59:40.919013         0.538893
17  2019-03-14 13:59:

subject path E:\Data\Data\rh\5\2019-04-15-14-24-43_1
subject and session name:  rh__5__2019-04-15-14-24-43_1
                     timeStamp  timeBetweenKeys
0   2019-04-15 14:24:53.235844         0.173810
1   2019-04-15 14:24:54.756426         0.460558
2   2019-04-15 14:24:56.333123         0.149292
3   2019-04-15 14:24:57.665697         0.284230
4   2019-04-15 14:24:58.897670         0.171744
5   2019-04-15 14:25:00.163694         0.205080
6   2019-04-15 14:25:01.541954         0.316591
7   2019-04-15 14:25:12.588585         0.206807
8   2019-04-15 14:25:13.920706         0.273255
9   2019-04-15 14:25:15.608497         0.626387
10  2019-04-15 14:25:17.507487         0.360110
11  2019-04-15 14:25:18.839868         0.270664
12  2019-04-15 14:25:20.149584         0.249178
13  2019-04-15 14:25:21.960203         0.360360
14  2019-04-15 14:25:26.745446         0.206739
15  2019-04-15 14:25:28.144305         0.340057
16  2019-04-15 14:25:30.108988         0.904410
17  2019-04-15 14:25:31.475

subject path E:\Data\Data\ys\1\2019-01-14-15-25-55_2
subject and session name:  ys__1__2019-01-14-15-25-55_2
                     timeStamp  timeBetweenKeys
0   2019-01-14 15:26:05.139689         0.352675
1   2019-01-14 15:26:05.850095         0.249688
2   2019-01-14 15:26:06.538280         0.238740
3   2019-01-14 15:26:07.271465         0.272714
4   2019-01-14 15:26:08.492277         0.326689
5   2019-01-14 15:26:09.392231         0.450604
6   2019-01-14 15:26:10.057992         0.206068
7   2019-01-14 15:26:11.012740         0.493397
8   2019-01-14 15:26:14.520560         0.359247
9   2019-01-14 15:26:15.268555         0.296612
10  2019-01-14 15:26:16.052541         0.326965
11  2019-01-14 15:26:16.853073         0.349842
12  2019-01-14 15:26:18.718523         0.949881
13  2019-01-14 15:26:19.406614         0.238083
14  2019-01-14 15:26:20.183326         0.326748
15  2019-01-14 15:26:21.039113         0.395283
16  2019-01-14 15:26:21.727025         0.225615
17  2019-01-14 15:26:22.460

subject path E:\Data\Data\ys\3\2019-01-17-15-05-1_1
subject and session name:  ys__3__2019-01-17-15-05-1_1
                     timeStamp  timeBetweenKeys
0   2019-01-17 15:05:11.178082         0.616475
1   2019-01-17 15:05:11.833963         0.305767
2   2019-01-17 15:05:12.499706         0.116705
3   2019-01-17 15:05:13.232719         0.370592
4   2019-01-17 15:05:14.075528         0.482395
5   2019-01-17 15:05:14.687586         0.250554
6   2019-01-17 15:05:15.331105         0.282434
7   2019-01-17 15:05:16.207733         0.518152
8   2019-01-17 15:05:19.216731         0.358922
9   2019-01-17 15:05:19.872088         0.307762
10  2019-01-17 15:05:20.694526         0.461732
11  2019-01-17 15:05:21.482771         0.405976
12  2019-01-17 15:05:22.069738         0.249204
13  2019-01-17 15:05:22.693311         0.271525
14  2019-01-17 15:05:23.380380         0.337027
15  2019-01-17 15:05:24.135016         0.393697
16  2019-01-17 15:05:24.791176         0.304537
17  2019-01-17 15:05:26.75814

subject path E:\Data\Data\ys\4_MS\2019-01-30-11-57-3_2
subject and session name:  ys__4_MS__2019-01-30-11-57-3_2
                     timeStamp  timeBetweenKeys
0   2019-01-30 11:57:21.296618         0.283352
1   2019-01-30 11:57:29.293126         0.193737
2   2019-01-30 11:57:36.476069         0.250575
3   2019-01-30 11:57:43.014331         0.339282
4   2019-01-30 12:00:01.812168         0.204978
5   2019-01-30 12:00:05.709019         0.339014
6   2019-01-30 12:00:07.685852         0.726816
7   2019-01-30 12:00:09.851775         0.505842
8   2019-01-30 12:00:23.962652         1.636098
9   2019-01-30 12:00:31.181048         0.306225
10  2019-01-30 12:00:34.945154         0.205863
11  2019-01-30 12:00:40.251648         0.482631
12  2019-01-30 12:00:51.123071         1.393313
13  2019-01-30 12:01:00.549094         1.194534
14  2019-01-30 12:01:01.681392         0.471157
15  2019-01-30 12:01:06.322796         0.315272
16  2019-01-30 12:01:10.030138         0.493012
17  2019-01-30 12:01:14